In [2]:
!pip install torchsummary
!pip install transformers
!pip install einops
!pip install typing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26305 sha256=ce6a13b567f90b833bba6e8c25b87a25b8feae460ff1f3da45ed6770a6a11c1c
  Stored in directory: /root/.cache/pip/wheels/9d/67/2f/53e3ef32ec48d11d7d60245255e2d71e908201d20c880c08ee
Successfully built typing


In [1]:

import torch
import transformers
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch import optim
import timm
from torchsummary import summary
from transformers import AutoModel, AutoTokenizer
from torch.functional import Tensor
import typing
import einops

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cuda


In [2]:
feat_ext = timm.create_model(
    'tf_efficientnet_b0.ns_jft_in1k',
    #'resnet18',
    pretrained=True,
    features_only=True,
    #out_indices = [2],
    in_chans = 1
)

class features_2D(nn.Module):
    def __init__ (self, channels, batch_size):
        super(features_2D, self).__init__()
        self.feat_ext = feat_ext
        self.channels = channels
        self.batch_size = batch_size
        self.channel_red = nn.Conv2d(in_channels=40,out_channels=channels,kernel_size=1)
        
    def forward(self, x:Tensor):
        x = einops.rearrange(x, 'b h w l -> (b h) w l')   # we concat the images in the batch along the height
        x = torch.unsqueeze(x, dim=1)                     #efficientnet (grayscale) needs a channel dimension of size 1
        x = self.feat_ext(x)[2]                           #we choose the layer from which we extract features
        x = self.channel_red(x)                           #1x1 convs help reduce the number of channels by mixing
        x = einops.rearrange(x, '(b h) c w l -> b h c w l', b=self.batch_size) #reverse the first step

        return x

model = features_2D(channels=1,batch_size=1)

In [ ]:

sample = torch.randn(1,224,224,224)

output = model(sample)
print(output.shape)

print(torch.cuda.memory_summary(device=None, abbreviated=False))

: 